In [112]:
import pandas as pd
import featuretools as ft
import numpy as np
from scipy import stats

In [113]:
pd.set_option('display.max_columns', 999)

In [114]:
df_sat = pd.read_csv("https://raw.githubusercontent.com/AVIMARCUS6/Personal_Work/master/2012_SAT_Results.csv", header = 0, sep = ",", index_col = "DBN")

df_school = pd.read_csv("https://raw.githubusercontent.com/AVIMARCUS6/Personal_Work/master/2014_-_2015_DOE_High_School_Directory.csv", header = 0, sep = ",", index_col = "dbn")

In [115]:
df_raw = pd.merge(df_sat, df_school, left_index=True, right_index=True)

In [116]:
df = df_raw.copy()

In [117]:
df.dtypes

SCHOOL NAME                          object
Num of SAT Test Takers               object
SAT Critical Reading Avg. Score      object
SAT Math Avg. Score                  object
SAT Writing Avg. Score               object
school_name                          object
borough                              object
building_code                        object
phone_number                         object
fax_number                           object
grade_span_min                      float64
grade_span_max                        int64
expgrade_span_min                   float64
expgrade_span_max                   float64
bus                                  object
subway                               object
primary_address_line_1               object
city                                 object
state_code                           object
postcode                              int64
website                              object
total_students                      float64
campus_name                     

In [118]:
# Drop Duplicate Columns
df = df.drop(['school_name', 'SCHOOL NAME'], axis = 1)

In [119]:
# drop column if 60% or more of column is Nan
def drop_majority(df):
    for i in df:
        if sum(df[i].isnull()) >= 0.6 * len(df):
            df.drop(i, axis=1, inplace=True)

In [120]:
drop_majority(df)

In [121]:
df.shape

(369, 52)

In [122]:
df = df[df['Num of SAT Test Takers'] != 's']

In [123]:
def informative(df):
    non_informative = [column for column in df.columns if len(df[column].unique()) == 1]
    informative_columns = list(set(df.columns.to_list()) - set(non_informative))
    return df[informative_columns]
    
def some_nans(df):
    informative_df = informative(df)
    some_nans_bools = pd.isnull(informative_df).any()
    return some_nans_bools.index[some_nans_bools]

def impute_mode(df):
    nan_cols = some_nans(df)
    col_mode = df[nan_cols].mode()
    col_mode = df[nan_cols].mode().iloc[0]
    imputed_mode_df = df.fillna(col_mode)
    return imputed_mode_df

def impute_means(df):
    nan_cols = some_nans(df)
    col_means = df[nan_cols].mean()
    print(col_means)
    imputed_df = df.fillna(col_means)
    return imputed_df

In [124]:
df = impute_mode(df)

In [125]:
df.shape

(339, 52)

In [126]:
def nas_sorted(df):
    '''Displays count of null values for each column'''     
    return df.isnull().sum().sort_values(ascending = False)

In [127]:
nas_sorted(df)

NTA                                 0
BBL                                 0
extracurricular_activities          0
advancedplacement_courses           0
language_classes                    0
program_highlights                  0
overview_paragraph                  0
campus_name                         0
total_students                      0
website                             0
postcode                            0
state_code                          0
city                                0
primary_address_line_1              0
subway                              0
bus                                 0
grade_span_max                      0
grade_span_min                      0
fax_number                          0
phone_number                        0
building_code                       0
borough                             0
SAT Writing Avg. Score              0
SAT Math Avg. Score                 0
SAT Critical Reading Avg. Score     0
psal_sports_boys                    0
psal_sports_

In [128]:
df['SAT Critical Reading Avg. Score'] = pd.to_numeric(df['SAT Critical Reading Avg. Score'])
df['Num of SAT Test Takers'] = pd.to_numeric(df['Num of SAT Test Takers'])
df['SAT Math Avg. Score'] = pd.to_numeric(df['SAT Math Avg. Score'])
df['SAT Writing Avg. Score'] = pd.to_numeric(df['SAT Writing Avg. Score'])


In [129]:
# After converting all columns to int or float, add new column with sum of SAT section scores for total SAT score
df['SAT_Avg_Score'] = df['SAT Writing Avg. Score'] + df['SAT Math Avg. Score'] + df['SAT Critical Reading Avg. Score']

In [130]:
df_categories = df.select_dtypes('object').astype('category')

In [134]:
X = df_categories.apply(lambda x: x.cat.codes)
X.head().to_numpy()

array([[  2,  68,  20,  24,  60, 118,  76,  15,   0,  14,  12,  61, 148,
         21, 244, 287, 207, 207,  79, 249,  32, 190,  34,  74,  17, 146,
         21,   0,  19,  32,   0,   0,   0,  51,  46,  24,  76,  59],
       [  2,  82,  72,  11, 154, 135,  66,  15,   0, 322,  12, 264, 331,
         30, 152, 203,  85,  85, 101, 120,  33, 206,  89, 229,  57,  84,
        118,   2,  13,  23,   0,   0,   1,  20,   8,  23,  67,  59],
       [  2,  69,  22,   9, 135,  89, 147,  15,   0, 199,  12, 277, 208,
         53, 175,  66, 179, 179,  49, 265,  71,  68,   0,  46, 134,  18,
        160,  59,  19,  32,   0,   0,   1,  51,  47,  23, 147,  40],
       [  2,  64,  23,  27,  57, 119,  43,  15,   0, 260,  12,  89,  20,
         43, 236, 292, 206, 206, 209, 170,  31, 197, 262,  50,  63,  22,
        153,  25,   8,  32,   0,   0,   0,  28,  45,  23,  43,  24],
       [  2,  63,  47,   8,  61, 138,  18,  15,   0, 274,  12,  98,   2,
         15,  34,  77, 145, 145,  16,   3,  71, 117, 251,   4, 160,

In [182]:
y = df.SAT_Avg_Score

In [184]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)

In [185]:
from sklearn.tree import DecisionTreeClassifier

In [186]:
model = DecisionTreeClassifier()

In [187]:
model.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [188]:
from sklearn import tree

In [189]:
from sklearn.tree import DecisionTreeClassifier

In [171]:
dtc = DecisionTreeClassifier()

In [192]:
from dtreeviz.trees import *
viz = dtreeviz(dtc,
               X_train,
               y_train,
               target_name='smart',
               feature_names=X.columns,
              )
              
viz

AttributeError: 'DecisionTreeClassifier' object has no attribute 'tree_'

In [172]:
from sklearn import tree
from IPython.display import SVG, display
from graphviz import Source 

In [174]:
from sklearn import tree
from IPython.display import SVG, display
from graphviz import Source 


graph_1 = Source(tree.export_graphviz(model, out_file=None,
                                feature_names=X.columns))

In [175]:
graph_1

In [176]:
from dtreeviz.trees import dtreeviz
viz = dtreeviz(dtc,
               X.to_numpy(),
               y,
               target_name='is_smart',
               feature_names=X.columns,
               class_names=["Pass", "Fail"]
              )

AttributeError: 'DecisionTreeClassifier' object has no attribute 'tree_'